<a href="https://colab.research.google.com/github/moch1996/Vaycold/blob/main/Recommend_Algorithm/%EC%9D%B4%EC%9B%83%EA%B8%B0%EB%B0%98_%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이웃기반 협업필터링
 - 협업필터링 종류 : 모델기반, 이웃기반, 하이브리드 딥러닝 기반 
 - 이웃기반 협업필터링 -> 유사도 계산, 아이템기반, 유저기반 ... 

## 라이브러리 불러오기

In [6]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [5]:
path = '/content/drive/MyDrive/fastcampus/recommend/movielens'
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [8]:
print(ratings_df.shape)
print(ratings_df.head())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [9]:
train_df, test_df = train_test_split(ratings_df, test_size = 0.2, random_state = 1234)

## Sparse matrix 만들기

 - sparse matrix = (user, movie)

In [11]:
# Sparse Matrix 만들기
# Sparse Matrix란 듬성듬성 있는 매트릭스임.
# pandas dataframe의 unstack 사용

In [20]:
print(train_df.shape)
print(train_df.head())

(80668, 4)
       userId  movieId  rating   timestamp
95713     600     5943     3.0  1237714356
61560     407     2571     5.0  1424349171
77204     482     8958     4.0  1105397126
93367     599     2322     2.5  1498515283
90892     590     2959     3.5  1258416553


In [17]:
sparse_matrix = train_df.groupby('movieId').apply(lambda x : pd.Series(x['rating'].values, index = x['userId'])).unstack()
sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,3.5,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,4.0,NaN,4.0,3.0,4.0,2.5,NaN,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,2.5,4.0,NaN,4.0,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# unstack을 적용안했을 시
sparse_matrix2 = train_df.groupby('movieId').apply(lambda x : pd.Series(x['rating'].values, index = x['userId']))
sparse_matrix2 # 해석을 해보자.
               # groupby('movieId) 를 하였으므로 movieId로 묶는 다는 소리임 그런데 어떻게 묶느냐?? 뒤에 apply함수를 이용해서 묶을거임.
               # apply 함수를 보자 
               

movieId  userId
1        46        5.0
         161       4.0
         453       5.0
         604       3.0
         169       4.5
                  ... 
193573   184       4.0
193579   184       3.5
193581   184       4.0
193587   184       3.5
193609   331       4.0
Length: 80668, dtype: float64

In [26]:
train_df[train_df.userId==46].head(2)

,userId,movieId,rating,timestamp
6876,46,1,5.0,834787906
6901,46,318,4.0,834787757


In [31]:
train_df[train_df.movieId==1].rating

6876     5.0
23663    4.0
70844    5.0
97043    3.0
24321    4.5
        ... 
81082    4.0
68337    4.0
71257    5.0
76227    3.0
19179    4.0
Name: rating, Length: 169, dtype: float64

In [33]:
train_df[train_df.movieId==1].rating.values

array([5. , 4. , 5. , 3. , 4.5, 3.5, 4. , 4.5, 4.5, 4. , 4. , 4.5, 3. ,
       4. , 3.5, 4. , 4. , 3.5, 3. , 4. , 4. , 3.5, 4. , 5. , 5. , 4.5,
       4. , 2. , 4. , 4. , 4. , 4.5, 2.5, 4.5, 4. , 5. , 4. , 3. , 3. ,
       2. , 4. , 4. , 5. , 4. , 2. , 5. , 4. , 3. , 0.5, 4. , 5. , 5. ,
       3.5, 4. , 3.5, 4. , 3. , 3. , 4. , 4.5, 5. , 4.5, 3. , 4. , 4. ,
       4. , 4. , 5. , 3.5, 3. , 3. , 5. , 5. , 4.5, 4. , 2.5, 2. , 2.5,
       5. , 4. , 2. , 2. , 4. , 5. , 4. , 3.5, 4. , 4. , 4.5, 4.5, 4. ,
       4. , 4. , 4. , 5. , 3. , 3. , 3. , 4. , 3.5, 4. , 5. , 4. , 4. ,
       4. , 5. , 5. , 4. , 5. , 3. , 4. , 5. , 4. , 3.5, 4. , 4. , 4. ,
       4. , 5. , 2.5, 4. , 3. , 3.5, 4. , 5. , 3. , 4. , 4. , 2.5, 4.5,
       5. , 5. , 5. , 5. , 3.5, 3.5, 4.5, 3. , 5. , 3.5, 4. , 3.5, 5. ,
       4. , 3. , 4. , 3. , 4. , 4. , 3. , 1.5, 3. , 2.5, 3. , 4. , 5. ,
       4.5, 5. , 4. , 4. , 5. , 5. , 4. , 3. , 4. , 4. , 5. , 3. , 4. ])

In [35]:
train_df[train_df.movieId==1].userId

6876      46
23663    161
70844    453
97043    604
24321    169
        ... 
81082    514
68337    443
71257    456
76227    480
19179    124
Name: userId, Length: 169, dtype: int64